In [1]:
import os 
import json
import time
import pandas as pd

from tqdm import tqdm
from langchain_mistralai import ChatMistralAI
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

# Генерация вопросов к случайным чанкам

In [2]:
api_key = os.environ.get("MISTRAL_KEY")
model = ChatMistralAI(model="mistral-large-latest", api_key=api_key)

In [3]:
def get_guestion_from_chunk(question):

    prompt = f"""
        Это отрывок текста, к которому нужно сформровать вопрос:
        `{question}`

        Теперь задай вопрос к тексту и верни только его
    """
    response = model.invoke(prompt)
    return response.content

In [4]:
df = pd.read_json("chunks/codex_df.jsonl", lines=True)

In [5]:
print(df["page_content"].iloc[0])

УК РФ Статья 1. Уголовное законодательство Российской Федерации

1. Уголовное законодательство Российской Федерации состоит из настоящего Кодекса. Новые законы, предусматривающие уголовную ответственность, подлежат включению в настоящий Кодекс.
2. Настоящий Кодекс основывается наКонституцииРоссийской Федерации и общепризнанных принципах и нормах международного права.


In [6]:
get_guestion_from_chunk(df["page_content"].iloc[0])

'Как формируется уголовное законодательство Российской Федерации?'

In [7]:
sampled_df = df.sample(100)

In [8]:
sampled_df.columns

Index(['id', 'metadata', 'page_content', 'type'], dtype='object')

In [9]:
lst = []

for x in tqdm(sampled_df['page_content'].to_list()):

    time.sleep(3)
    answer = get_guestion_from_chunk(x)
    lst.append(answer)

100%|██████████| 100/100 [06:48<00:00,  4.08s/it]


In [10]:
sampled_df["question"] = lst
sampled_df.head()

,id,metadata,page_content,type,question
1756,NaN,"{'name_codex': 'Уголовный кодекс (УК РФ)', 'ur...",наказывается штрафом в размере от ста тысяч до...,Document,Какое наказание предусмотрено за данное правон...
880,NaN,"{'name_codex': 'Уголовный кодекс (УК РФ)', 'ur...",наказывается штрафом в размере до одного милли...,Document,Какое наказание предусмотрено за данное правон...
527,NaN,"{'name_codex': 'Уголовный кодекс (УК РФ)', 'ur...",наказываются лишением свободы на срок от четыр...,Document,Какие действия наказываются лишением свободы н...
1605,NaN,"{'name_codex': 'Уголовный кодекс (УК РФ)', 'ur...","1. Деяния, направленные на организацию занятия...",Document,"Какое наказание предусмотрено за деяния, напра..."
1316,NaN,"{'name_codex': 'Уголовный кодекс (УК РФ)', 'ur...",наказываются лишением свободы на срок от двена...,Document,Какое наказание предусмотрено за данное престу...


# Ретрив 

такая же функция как в пайплайне, только сделал выдачу более удобную под задачу

In [13]:
FAISS_DB = "C:/Users/Kika-/Desktop/itmo_slivs/llm_project/FinanceInsight/data/index"

EMBEDDING_MODEL_NAME = 'intfloat/multilingual-e5-small'

embedding_model = HuggingFaceEmbeddings(
    model_name = EMBEDDING_MODEL_NAME,
    multi_process = True,
    encode_kwargs = {"normalize_embeddings": True}
)

vector_store = FAISS.load_local(
    FAISS_DB, 
    embedding_model, 
    allow_dangerous_deserialization = True
)

def retrieve_v1(query: str):
    """
    Получение ответа на вопросы об экономике, финансах, законах.
    Данные получены из статей, сайтов, 

    Args:
        query: вопрос пользователя.
    """
    
    retrieved_docs = vector_store.similarity_search(query = query, k=2)
    
    return [x.page_content for x in retrieved_docs] 

In [14]:
retrieved_docs_v1 = []
retrieved_docs_v2 = []

for x in tqdm(sampled_df["question"].to_list()):

    retrieved = retrieve_v1(x)
    retrieved_docs_v1.append(retrieved[0])
    retrieved_docs_v2.append(retrieved[1])

100%|██████████| 100/100 [50:57<00:00, 30.58s/it]


In [15]:
sampled_df["retrieved_docs_v1"] = retrieved_docs_v1
sampled_df["retrieved_docs_v2"] = retrieved_docs_v2

# Метрики

In [16]:
def recall_at_k(retrieved_docs, relevant_docs, k):
    retrieved_at_k = retrieved_docs[:k]
    relevant_in_k = len(set(retrieved_at_k) & set(relevant_docs))
    return relevant_in_k / len(relevant_docs) if relevant_docs else 0.0

def precision_at_k(retrieved_docs, relevant_docs, k):
    retrieved_at_k = retrieved_docs[:k]
    relevant_in_k = len(set(retrieved_at_k) & set(relevant_docs))
    return relevant_in_k / k if k > 0 else 0.0

In [17]:
precision_at_1 = []
recall_at_2 = []

for x in tqdm(range(sampled_df.shape[0])):

    relevant_docs = [sampled_df["page_content"].iloc[x]]
    retrieved_docs = [sampled_df["retrieved_docs_v1"].iloc[x], sampled_df["retrieved_docs_v2"].iloc[x]]
    temp_recall = recall_at_k(retrieved_docs, relevant_docs, k=2)
    temp_precision = precision_at_k(retrieved_docs, relevant_docs, k=1)

    precision_at_1.append(temp_precision)
    recall_at_2.append(temp_recall)

100%|██████████| 100/100 [00:00<00:00, 12528.54it/s]


In [18]:
avg_precision_at_1 = sum(precision_at_1) / len(precision_at_1)
avg_recall_at_2 = sum(recall_at_2) / len(recall_at_2)

In [19]:
print("Средний precision@1 :", avg_precision_at_1)
print("Средний recall@2 :", avg_recall_at_2)

Средний precision@1 : 0.59
Средний recall@2 : 0.62
